In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import to_date, year, month, dayofmonth, dayofweek, hour
from pyspark.sql.functions import countDistinct, avg, col
from pyspark.sql import Window
import math

In [2]:
spark = SparkSession.builder \
    .appName("DivvyBikes_EDA") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

In [3]:
schema = StructType([
    StructField("ride_id", StringType(), True),
    StructField("rideable_type", StringType(), True),
    StructField("started_at", TimestampType(), True),
    StructField("ended_at", TimestampType(), True),
    StructField("start_station_name", StringType(), True),
    StructField("start_station_id", StringType(), True),
    StructField("end_station_name", StringType(), True),
    StructField("end_station_id", StringType(), True),
    StructField("start_lat", DoubleType(), True),
    StructField("start_lng", DoubleType(), True),
    StructField("end_lat", DoubleType(), True),
    StructField("end_lng", DoubleType(), True),
    StructField("member_casual", StringType(), True)
])

# Загружаем данные
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .option("timestampFormat", "yyyy-MM-dd HH:mm:ss") \
    .option("encoding", "UTF-8") \
    .option("mode", "DROPMALFORMED") \
    .csv("output.csv")
df_with_features = df.withColumn("ride_date", to_date(col("started_at"))) \
    .withColumn("ride_year", year(col("started_at"))) \
    .withColumn("ride_month", month(col("started_at"))) \
    .withColumn("ride_day", dayofmonth(col("started_at"))) \
    .withColumn("ride_dayofweek", dayofweek(col("started_at"))) \
    .withColumn("ride_hour", hour(col("started_at"))) \
    .withColumn("ride_duration_seconds", 
               (col("ended_at").cast("long") - col("started_at").cast("long"))) \
    .withColumn("ride_duration_minutes", 
               (col("ended_at").cast("long") - col("started_at").cast("long")) / 60.0)

# Функция для расчета расстояния (формула Гаверсинуса)
def haversine_distance(lat1, lon1, lat2, lon2):
    """Расчет расстояния между двумя точками в км"""
    R = 6371  # Радиус Земли в км
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    return R * c

# Регистрируем UDF
haversine_udf = udf(haversine_distance, DoubleType())

df_with_geo_features = df_with_features \
    .withColumn("distance_km", 
                haversine_udf(col("start_lat"), col("start_lng"), col("end_lat"), col("end_lng"))) \
    .withColumn("lat_grid", round(col("start_lat"), 2)) \
    .withColumn("lng_grid", round(col("start_lng"), 2)) \
    .withColumn("is_weekend", 
                when((col("ride_dayofweek") == 1) | 
                     (col("ride_dayofweek") == 7), True).otherwise(False)) \
    .withColumn("lat_diff", abs(col("end_lat") - col("start_lat"))) \
    .withColumn("lng_diff", abs(col("end_lng") - col("start_lng"))) \
    .withColumn("coord_diff", col("lat_diff") + col("lng_diff")) \
    .withColumn("same_station", 
                when(col("start_station_name") == col("end_station_name"), True).otherwise(False))


In [4]:
# Пробуем упрощенный подход без проверки скорости
filtered_simple = df_with_geo_features.filter(
    # 1. Базовая валидация
    (col("ride_id").isNotNull()) &
    (col("started_at").isNotNull()) &
    (col("ended_at").isNotNull()) &
    (col("member_casual").isNotNull()) &
    (col("rideable_type").isNotNull()) &
    
    # 2. Даты
    (col("ended_at") > col("started_at")) &
    
    # 3. Длительность
    (col("ride_duration_minutes") >= 1) &
    (col("ride_duration_minutes") <= 1440) &
    
    # 4. Координаты
    (col("start_lat").between(41.6, 42.1)) &
    (col("start_lng").between(-87.95, -87.5)) &
    (col("end_lat").between(41.6, 42.1)) &
    (col("end_lng").between(-87.95, -87.5)) &
    
    # 5. Не нулевые координаты
    (col("start_lat") != 0) &
    (col("start_lng") != 0) &
    
    # 6. Не тестовые станции
    (~lower(col("start_station_name")).contains("test")) &
    (~lower(col("start_station_name")).contains("hubbard")) &
    (~lower(col("start_station_name")).contains("watson"))
)

# Удаляем дубликаты
filtered_simple = filtered_simple.dropDuplicates(["ride_id"])

# Используем этот DataFrame для дальнейшей работы
filtered_df = filtered_simple

 Осталось: 18,455,642 записей


In [5]:
filtered_df = filtered_df.withColumn(
    "start_geo_hash", 
    concat(round(col("start_lat"), 3).cast("string"), lit("_"), round(col("start_lng"), 3).cast("string"))
).withColumn(
    "end_geo_hash", 
    concat(round(col("end_lat"), 3).cast("string"), lit("_"), round(col("end_lng"), 3).cast("string"))
)

In [6]:
# Функция для заполнения пропущенных станций
def fill_missing_stations(df, station_col, geo_hash_col):
    """Заполняет пропущенные названия станций наиболее частыми в той же геозоне"""
    
    # Создаем оконную функцию для нахождения наиболее частой станции в геозоне
    window_spec = Window.partitionBy(geo_hash_col).orderBy(desc("count"))
    
    # Создаем DataFrame с наиболее частыми станциями
    common_stations = df.filter(col(station_col).isNotNull()) \
        .groupBy(geo_hash_col, station_col) \
        .agg(count("*").alias("count")) \
        .withColumn("rank", row_number().over(window_spec)) \
        .filter(col("rank") == 1) \
        .select(geo_hash_col, col(station_col).alias(f"common_{station_col}"))
    
    # Присоединяем наиболее частые станции
    df_filled = df.join(common_stations, on=geo_hash_col, how="left") \
        .withColumn(f"{station_col}_clean", 
                   coalesce(col(station_col), col(f"common_{station_col}"))) \
        .drop(f"common_{station_col}")
    
    return df_filled

# Заполняем пропущенные стартовые станции
filtered_df = fill_missing_stations(filtered_df, "start_station_name", "start_geo_hash")

# Заполняем пропущенные конечные станции
filtered_df = fill_missing_stations(filtered_df, "end_station_name", "end_geo_hash")

# Заменяем оригинальные колонки очищенными
filtered_df = filtered_df \
    .drop("start_station_name", "end_station_name") \
    .withColumnRenamed("start_station_name_clean", "start_station_name") \
    .withColumnRenamed("end_station_name_clean", "end_station_name") \
    .drop("start_geo_hash", "end_geo_hash")


In [7]:
# 1. Удаляем поездки с экстремально малым расстоянием при большой длительности
filtered_df = filtered_df.filter(
    ~((col("distance_km") < 0.01) & (col("ride_duration_minutes") > 40))
)

# 2. Фильтр для поездок на ту же станцию с аномальной длительностью
filtered_df = filtered_df.filter(
    ~(
        (col("start_station_name") == col("end_station_name")) &
        (col("ride_duration_minutes") > 1440)
    )
)


# 4. Удаляем поездки с нереальной скоростью
filtered_df = filtered_df.withColumn(
    "speed_kmh",
    when(col("ride_duration_minutes") > 0,
         col("distance_km") / (col("ride_duration_minutes") / 60.0)).otherwise(0)
)

filtered_df = filtered_df.filter(
    (col("speed_kmh") <= 45) | (col("speed_kmh").isNull())
)


In [8]:
cleaned_df_with_features = filtered_df \
    .withColumn("lat_grid", round(col("start_lat"), 2)) \
    .withColumn("lng_grid", round(col("start_lng"), 2)) \
    .withColumn("is_weekend", 
                when((col("ride_dayofweek") == 1) | 
                     (col("ride_dayofweek") == 7), True).otherwise(False)) \
    .withColumn("lat_diff", abs(col("end_lat") - col("start_lat"))) \
    .withColumn("lng_diff", abs(col("end_lng") - col("start_lng"))) \
    .withColumn("coord_diff", col("lat_diff") + col("lng_diff")) \
    .withColumn("speed_kmh", 
                when(col("ride_duration_minutes") > 0, 
                     col("distance_km") / (col("ride_duration_minutes") / 60.0)).otherwise(0))

In [10]:
# Разделяем проверку на несколько частей

# Часть 1: Базовая статистика
print("\n1. БАЗОВАЯ СТАТИСТИКА:")
print()

basic_stats = cleaned_df_with_features.select(
    count("*").alias("total_records"),
    round(mean("ride_duration_minutes"), 2).alias("avg_duration"),
    round(min("ride_duration_minutes"), 2).alias("min_duration"),
    round(max("ride_duration_minutes"), 2).alias("max_duration")
).collect()[0]

print(f"  Всего записей: {basic_stats['total_records']:,}")
print(f"  Средняя длительность: {basic_stats['avg_duration']} мин")
print(f"  Минимальная: {basic_stats['min_duration']} мин")
print(f"  Максимальная: {basic_stats['max_duration']} мин")

# Часть 2: Пропущенные значения
print("\n2. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ:")
print()

missing_stats = cleaned_df_with_features.select(
    count(when(col("start_station_name").isNull(), 1)).alias("missing_start_stations"),
    count(when(col("end_station_name").isNull(), 1)).alias("missing_end_stations"),
    count(when(col("start_lat").isNull(), 1)).alias("missing_start_lat"),
    count(when(col("end_lat").isNull(), 1)).alias("missing_end_lat")
).collect()[0]

total_records = basic_stats['total_records']
print(f"  Стартовые станции: {missing_stats['missing_start_stations']:,} "
      f"({missing_stats['missing_start_stations']/total_records*100:.4f}%)")
print(f"  Конечные станции: {missing_stats['missing_end_stations']:,} "
      f"({missing_stats['missing_end_stations']/total_records*100:.4f}%)")

# Часть 3: География
print("\n3. ГЕОГРАФИЧЕСКИЙ ДИАПАЗОН:")
print()

geo_stats = cleaned_df_with_features.select(
    round(min("start_lat"), 2).alias("min_start_lat"),
    round(max("start_lat"), 2).alias("max_start_lat"),
    round(min("start_lng"), 2).alias("min_start_lng"),
    round(max("start_lng"), 2).alias("max_start_lng")
).collect()[0]

print(f"  Широта: [{geo_stats['min_start_lat']:.2f}, {geo_stats['max_start_lat']:.2f}]")
print(f"  Долгота: [{geo_stats['min_start_lng']:.2f}, {geo_stats['max_start_lng']:.2f}]")

# Часть 4: Типы данных (быстрая проверка)
print("\n4. РАСПРЕДЕЛЕНИЕ ПО ТИПАМ:")
print()

# Используем приближенную проверку через выборку
sample_df = cleaned_df_with_features.sample(0.01)  # 1% выборка

bike_types = sample_df.select("rideable_type").distinct().collect()
user_types = sample_df.select("member_casual").distinct().collect()

print(f"  Типы велосипедов: {[row['rideable_type'] for row in bike_types]}")
print(f"  Типы пользователей: {[row['member_casual'] for row in user_types]}")

# Часть 5: Дополнительная статистика по подвыборке
print("\n5. СТАТИСТИКА ПО ВЫБОРКЕ (1%):")
print()

sample_stats = sample_df.select(
    count("*").alias("sample_size"),
    round(mean("distance_km"), 3).alias("avg_distance_sample"),
    round(stddev("distance_km"), 3).alias("std_distance_sample"),
    round(mean("speed_kmh"), 1).alias("avg_speed_sample")
).collect()[0]

print(f"  Размер выборки: {sample_stats['sample_size']:,}")
print(f"  Среднее расстояние: {sample_stats['avg_distance_sample']} км")
print(f"  Средняя скорость: {sample_stats['avg_speed_sample']} км/ч")


1. БАЗОВАЯ СТАТИСТИКА:



Py4JJavaError: An error occurred while calling o723.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 248) (192.168.1.67 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:252)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:143)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:158)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:178)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:261)
	at org.apache.spark.sql.execution.python.BatchEvalPythonEvaluatorFactory.evaluate(BatchEvalPythonExec.scala:83)
	at org.apache.spark.sql.execution.python.EvalPythonEvaluatorFactory$EvalPythonPartitionEvaluator.eval(EvalPythonEvaluatorFactory.sc